<a href="https://colab.research.google.com/github/SaranshRegmi10/ADD-CONTACT/blob/main/Tumor_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2025/05/19 -- 05/20 -- 05/21 -- 05/22

In [2]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import nibabel as nb
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, Dropout, concatenate, Input
from tensorflow.keras.models import Model,load_model
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

In [4]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("awsaf49/brats20-dataset-training-validation")

print("Path to dataset files:", path)
print(os.listdir(path))

Path to dataset files: /kaggle/input/brats20-dataset-training-validation
['BraTS2020_ValidationData', 'BraTS2020_TrainingData']


In [5]:
dir_path = "BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"
base_path = os.path.join(path,dir_path)
slice_index = 75

In [ ]:
mobilities = ['t1','t2','flair','t1ce']
X_data = []
Y_data = []

for subject_folder in os.listdir(base_path):
    subject_path = os.path.join(base_path, subject_folder)
    if not os.path.isdir(subject_path) or not subject_folder.startswith("BraTS20_Training_"):
        continue

    try:
        # Load all modalities once
        vols = {}
        for mod in mobilities:
            file_path = os.path.join(subject_path, f"{subject_folder}_{mod}.nii")
            vols[mod] = nb.load(file_path).get_fdata()

        mask_path = os.path.join(subject_path, f"{subject_folder}_seg.nii")
        seg = nb.load(mask_path).get_fdata()

        # Loop through multiple slices
        for slice_index in range(60, 91):  # 60 to 90
            channels = []
            skip = False

            for mod in mobilities:
                slice_ = vols[mod][:, :, slice_index]
                resized = cv2.resize(slice_, (128, 128))
                norm = (resized - np.min(resized)) / (np.max(resized) - np.min(resized) + 1e-8)
                channels.append(norm)

            x = np.stack(channels, axis=-1)

            mask_slice = seg[:, :, slice_index]
            if np.max(mask_slice) == 0:
                continue  # skip slices with no tumor

            mask_resized = cv2.resize(mask_slice, (128, 128), interpolation=cv2.INTER_NEAREST)
            y = (mask_resized > 0).astype(np.float32)[..., np.newaxis]

            X_data.append(x)
            Y_data.append(y)

    except Exception as e:
        print(f"Skipping {subject_folder}: {e}")

X_data = np.array(X_data)
Y_data = np.array(Y_data)
print("Final shape:", X_data.shape, Y_data.shape)

Skipping BraTS20_Training_355: No such file or no access: '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/BraTS20_Training_355_seg.nii'


In [ ]:
print(X_data.shape)
print(Y_data.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

# U-net

In [ ]:
def unet_model(hp):
    inputs = Input(shape=(128, 128, 4))

    # Tune dropout
    dropout_rate = hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)

    # Tune number of filters
    f1 = hp.Choice('filters_layer1', values=[16, 32, 64])

    # Encoder
    c1 = Conv2D(f1, 3, activation='relu', padding='same')(inputs)
    c1 = Dropout(dropout_rate)(c1)
    c1 = Conv2D(f1, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D()(c1)

    f2 = f1 * 2
    c2 = Conv2D(f2, 3, activation='relu', padding='same')(p1)
    c2 = Dropout(dropout_rate)(c2)
    c2 = Conv2D(f2, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D()(c2)

    f3 = f2 * 2
    c3 = Conv2D(f3, 3, activation='relu', padding='same')(p2)
    c3 = Dropout(dropout_rate)(c3)
    c3 = Conv2D(f3, 3, activation='relu', padding='same')(c3)
    p3 = MaxPooling2D()(c3)

    # Bottleneck
    f4 = f3 * 2
    c4 = Conv2D(f4, 3, activation='relu', padding='same')(p3)
    c4 = Dropout(dropout_rate)(c4)
    c4 = Conv2D(f4, 3, activation='relu', padding='same')(c4)

    # Decoder
    u1 = Conv2DTranspose(f3, 2, strides=2, padding='same')(c4)
    u1 = concatenate([u1, c3])
    c5 = Conv2D(f3, 3, activation='relu', padding='same')(u1)
    c5 = Conv2D(f3, 3, activation='relu', padding='same')(c5)

    u2 = Conv2DTranspose(f2, 2, strides=2, padding='same')(c5)
    u2 = concatenate([u2, c2])
    c6 = Conv2D(f2, 3, activation='relu', padding='same')(u2)
    c6 = Conv2D(f2, 3, activation='relu', padding='same')(c6)

    u3 = Conv2DTranspose(f1, 2, strides=2, padding='same')(c6)
    u3 = concatenate([u3, c1])
    c7 = Conv2D(f1, 3, activation='relu', padding='same')(u3)
    c7 = Conv2D(f1, 3, activation='relu', padding='same')(c7)

    outputs = Conv2D(1, 1, activation='sigmoid')(c7)

    model = Model(inputs=[inputs], outputs=[outputs])

    # Tune learning rate
    lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])

    return model


--break point --

In [ ]:
tuner = kt.RandomSearch(
    unet_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
)

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_split=0.2)

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

best_learning_rate = best_hps.get('learning_rate')
best_dropout = best_hps.get('dropout')
# print(f"Best dropout: {best_dropout}")
print(f"Best learning rate: {best_learning_rate}")

In [ ]:
def unet_final_model(learning_rate,dropout_rate):

  input = Input((128,128,4))

  #Encoder
  c1 = Conv2D(16,(3,3),activation="relu",padding="same")(input)
  c1 = Dropout(dropout_rate)(c1)
  c1 = Conv2D(16,(3,3),activation="relu",padding="same")(c1)
  p1 = MaxPooling2D((2,2))(c1)

  c2 = Conv2D(32,(3,3),activation="relu",padding= "same")(p1)
  c2 = Dropout(dropout_rate)(c2)
  c2 = Conv2D(32,(3,3),activation="relu",padding="same")(c2)
  p2 = MaxPooling2D((2,2))(c2)

  c3 = Conv2D(64,(3,3),activation="relu",padding="same")(p2)
  c3 = Dropout(dropout_rate)(c3)
  c3 = Conv2D(64,(3,3),activation="relu",padding="same")(c3)
  p3 = MaxPooling2D((2,2))(c3)

  c4 = Conv2D(128,(3,3),activation="relu",padding="same")(p3)
  c4 = Dropout(dropout_rate)(c4)
  c4 = Conv2D(128,(3,3),activation="relu",padding="same")(c4)
  p4 = MaxPooling2D((2,2))(c4)

  #Bottlefask
  c5 = Conv2D(256,(3,3),activation="relu",padding="same")(p4)
  c5 = Dropout(dropout_rate)(c5)
  c5 = Conv2D(256,(3,3),activation="relu",padding="same")(c5)

  #Decoder
  u1 = Conv2DTranspose(128,(2,2),strides=(2,2),padding="same")(c5)
  u1 = concatenate([u1,c4])
  c6 = Conv2D(128,(3,3),activation="relu",padding="same")(u1)

  u2 = Conv2DTranspose(64,(2,2),strides=(2,2),padding="same")(c6)
  u2 = concatenate([u2,c3])
  c7 = Conv2D(64,(3,3),activation="relu",padding="same")(u2)

  u3 = Conv2DTranspose(32,(2,2),strides=(2,2),padding="same")(c7)
  u3 = concatenate([u3,c2])
  c8 = Conv2D(32,(3,3),activation="relu",padding="same")(u3)

  u4 = Conv2DTranspose(16,(2,2),strides=(2,2),padding="same")(c8)
  u4 = concatenate([u4,c1])
  c9 = Conv2D(16,(3,3),activation="relu",padding="same")(u4)

  output = Conv2D(1,(1,1),activation="sigmoid")(c9)

  model = Model(inputs=input,outputs=output )
  model.compile(optimizer=Adam(learning_rate=learning_rate),loss="binary_crossentropy",metrics=["accuracy"])

  return model

In [ ]:
model = unet_final_model(
  learning_rate = best_learning_rate,
  dropout_rate = best_dropout
)

In [ ]:
fitting = model.fit(X_train,y_train,validation_split=0.2,batch_size=8,epochs=20)

In [ ]:
model_path = 'model.h5'

In [ ]:
if os.path.exists(model_path):
    model = load_model(model_path, compile=False)
    print("✅ Trained model loaded successfully!")
else:
    raise FileNotFoundError("❌ Saved model 'model.h5' not found. Please train and save the model first.")

In [ ]:
# Predict on validation/test data
# For example, use the first 10 samples
n = 15
preds = model.predict(X_data[:n])

# Threshold predictions to make them binary (0 or 1)
preds_thresholded = (preds > 0.5).astype(np.uint8)

# Plot each sample
for i in range(n):
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))

    # Input image (let’s take the first channel for display)
    axs[0].imshow(X_data[i][:, :, 0], cmap='gray')
    axs[0].set_title("Input (FLAIR)")
    axs[0].axis('off')

    # Ground Truth
    axs[1].imshow(Y_data[i][:, :, 0], cmap='Reds')
    axs[1].set_title("Ground Truth Mask")
    axs[1].axis('off')

    # Prediction
    axs[2].imshow(preds_thresholded[i][:, :, 0], cmap='Reds')
    axs[2].set_title("Predicted Mask")
    axs[2].axis('off')

    plt.tight_layout()
    plt.show()

    plt.tight_layout()
    plt.savefig("segmentation_result.png")
    plt.show()

In [ ]:
if 'model' in locals() and isinstance(model, tf.keras.Model):
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_learning_rate = best_hps.get('learning_rate')

    model.compile(optimizer=Adam(learning_rate=best_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])


loss, acc = model.evaluate(X_test, y_test)
print(f"🎯 Test Accuracy: {acc:.4f}")

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(fitting.history['accuracy'], label='Train Accuracy') # Updated to use 'accuracy'
plt.plot(fitting.history['val_accuracy'], label='Val Accuracy') # Updated to use 'val_accuracy'
plt.xlabel("Epochs")
plt.ylabel("Accuracy") # Updated label
plt.title("Accuracy Over Epochs") # Updated title
plt.legend()
plt.grid(True)
plt.savefig("accuracy_graph.png")  # Updated filename
plt.show()

In [ ]:
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coefficient])


In [ ]:
def iou(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

# Evaluate the model on the test data using the new metrics
loss, dice, iou_score = model.evaluate(X_test, y_test)

print(f"🎯 Test Loss: {loss:.4f}")
print(f"🎯 Test Dice Coefficient: {dice:.4f}")
print(f"🎯 Test IoU: {iou_score:.4f}")